In [1]:
# Automatic Differentiation

In [2]:
import torch

print(torch.__version__)

1.3.0


In [3]:
### 上一节介绍的Tensor是这个包的核心类，
### 如果将其属性.requires_grad设置为True，
### 它将开始追踪(track)在其上的所有操作。
### 完成计算后，可以调用.backward()来完成所有梯度计算。此Tensor的梯度将累积到.grad属性中。